# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
weather_df = pd.read_csv("weather.csv")
weather_df

,City,Temp,Lat,Lon,Hum,Cloud,Wind,Country
0,Kidal,34.61,18.44,1.41,3.0,0.0,4.31,ML
1,Bluff,7.22,-46.60,168.33,79.0,0.0,0.45,NZ
2,New norfolk,14.97,-42.78,147.06,67.0,75.0,4.60,AU
3,Ca mau,25.23,9.18,105.15,78.0,17.0,2.72,VN
4,Yellowknife,-24.34,62.46,-114.35,83.0,40.0,3.60,CA
...,...,...,...,...,...,...,...,...
582,Valleyview,-5.45,55.07,-117.29,83.0,39.0,3.34,CA
583,Haibowan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
584,Chimbote,22.45,-9.09,-78.58,80.0,95.0,1.27,PE
585,Oil city,0.67,41.43,-79.71,90.0,1.0,3.60,US


In [18]:
weather_clean = weather_df.dropna()
weather_clean

,City,Temp,Lat,Lon,Hum,Cloud,Wind,Country
0,Kidal,34.61,18.44,1.41,3.0,0.0,4.31,ML
1,Bluff,7.22,-46.60,168.33,79.0,0.0,0.45,NZ
2,New norfolk,14.97,-42.78,147.06,67.0,75.0,4.60,AU
3,Ca mau,25.23,9.18,105.15,78.0,17.0,2.72,VN
4,Yellowknife,-24.34,62.46,-114.35,83.0,40.0,3.60,CA
...,...,...,...,...,...,...,...,...
581,Karratha,30.13,-20.74,116.85,76.0,25.0,5.08,AU
582,Valleyview,-5.45,55.07,-117.29,83.0,39.0,3.34,CA
584,Chimbote,22.45,-9.09,-78.58,80.0,95.0,1.27,PE
585,Oil city,0.67,41.43,-79.71,90.0,1.0,3.60,US


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
gmaps.configure(api_key=g_key)

In [21]:
# Store latitude and longitude in locations
# Fill NaN values and convert to float
#df['myCol'] = df['myCol'].astype('Int64')

locations = weather_clean[["Lat", "Lon"]]
humidity = weather_clean['Hum']

# Plot Heatmap
humidity_map = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
humidity_map.add_layer(heat_layer)
# Display figure
humidity_map


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

* Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [22]:
#filterinfDataframe = dfObj[(dfObj['Sale'] > 30) & (dfObj['Sale'] < 33) ]
ideal_weather = weather_clean[(weather_clean['Temp'] < 35) & (weather_clean['Temp'] > 15) & (weather_clean['Hum'] < 20) & (weather_clean['Cloud'] < 50)] 
ideal_weather

,City,Temp,Lat,Lon,Hum,Cloud,Wind,Country
0,Kidal,34.61,18.44,1.41,3.0,0.0,4.31,ML
159,Dabat,27.22,12.98,37.77,16.0,20.0,5.37,ET
253,Matay,28.22,28.42,30.78,9.0,0.0,1.88,EG
278,Tessalit,34.29,20.20,1.01,4.0,0.0,4.35,ML
307,Awbari,23.92,26.59,12.78,17.0,7.0,7.72,LY
369,Bikaner,29.45,28.02,73.30,14.0,40.0,2.13,IN
447,Tiznit,29.34,29.58,-9.50,15.0,0.0,1.22,MA
541,Donskoye,20.14,45.46,41.97,19.0,8.0,2.00,RU
560,Pathardi,31.66,19.17,75.18,11.0,0.0,4.06,IN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
hotel_df = ideal_weather.copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Temp,Lat,Lon,Hum,Cloud,Wind,Country,Hotel Name
0,Kidal,34.61,18.44,1.41,3.0,0.0,4.31,ML,
159,Dabat,27.22,12.98,37.77,16.0,20.0,5.37,ET,
253,Matay,28.22,28.42,30.78,9.0,0.0,1.88,EG,
278,Tessalit,34.29,20.20,1.01,4.0,0.0,4.35,ML,
307,Awbari,23.92,26.59,12.78,17.0,7.0,7.72,LY,
369,Bikaner,29.45,28.02,73.30,14.0,40.0,2.13,IN,
447,Tiznit,29.34,29.58,-9.50,15.0,0.0,1.22,MA,
541,Donskoye,20.14,45.46,41.97,19.0,8.0,2.00,RU,
560,Pathardi,31.66,19.17,75.18,11.0,0.0,4.06,IN,


In [24]:

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_search = "Hotel"
target_radius = 100000

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lon = row['Lon']
    latlon = f"{lat},{lon}"
    
    params = {
        "location" : latlon,
        "keyword": target_search,
        "radius": target_radius,
        "key": g_key
    }

    response = requests.get(base_url, params=params)
    hotel_data = response.json()
    
    try:
        #print(hotel_data['results'][0]['name'])
        hotel_df.loc[index,'Hotel Name'] = hotel_data['results'][0]['name']
                #hotel_df = hotel_df.append({hotel_data["results"][0]["name"]})
    except (KeyError, IndexError):
        hotel_df.loc[index,'Hotel Name'] = 'NaN'
        print("Missing result...skipping.")
    print('-------')


Missing result...skipping.
-------
-------
-------
Missing result...skipping.
-------
-------
-------
-------
-------
-------


In [25]:
hotel_df

,City,Temp,Lat,Lon,Hum,Cloud,Wind,Country,Hotel Name
0,Kidal,34.61,18.44,1.41,3.0,0.0,4.31,ML,NaN
159,Dabat,27.22,12.98,37.77,16.0,20.0,5.37,ET,Haile Resort Gondar
253,Matay,28.22,28.42,30.78,9.0,0.0,1.88,EG,فندق عمر الخيام المنيا - Omar El-Khayam Al Min...
278,Tessalit,34.29,20.20,1.01,4.0,0.0,4.35,ML,NaN
307,Awbari,23.92,26.59,12.78,17.0,7.0,7.72,LY,فندق أوباري
369,Bikaner,29.45,28.02,73.30,14.0,40.0,2.13,IN,The Laxmi Niwas Palace
447,Tiznit,29.34,29.58,-9.50,15.0,0.0,1.22,MA,Hotel Des Touristes
541,Donskoye,20.14,45.46,41.97,19.0,8.0,2.00,RU,"PARK HOTEL, a hotel complex"
560,Pathardi,31.66,19.17,75.18,11.0,0.0,4.06,IN,Hotel Raj Palace


In [26]:
#drop the cities that have no hotels within specified radius
blanks = hotel_df[hotel_df['Hotel Name'] == 'NaN'].index
hotel_df.drop(blanks, inplace=True)
hotel_df

,City,Temp,Lat,Lon,Hum,Cloud,Wind,Country,Hotel Name
159,Dabat,27.22,12.98,37.77,16.0,20.0,5.37,ET,Haile Resort Gondar
253,Matay,28.22,28.42,30.78,9.0,0.0,1.88,EG,فندق عمر الخيام المنيا - Omar El-Khayam Al Min...
307,Awbari,23.92,26.59,12.78,17.0,7.0,7.72,LY,فندق أوباري
369,Bikaner,29.45,28.02,73.30,14.0,40.0,2.13,IN,The Laxmi Niwas Palace
447,Tiznit,29.34,29.58,-9.50,15.0,0.0,1.22,MA,Hotel Des Touristes
541,Donskoye,20.14,45.46,41.97,19.0,8.0,2.00,RU,"PARK HOTEL, a hotel complex"
560,Pathardi,31.66,19.17,75.18,11.0,0.0,4.06,IN,Hotel Raj Palace


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [29]:
# Add marker layer ontop of heat map
# Display Map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
humidity_map.add_layer(markers)
humidity_map

Figure(layout=FigureLayout(height='420px'))

Looks like we're going to Africa or the Middle East!